## Preambule

In [1]:
# General packages
import numpy as np
import pandas as pd
from tqdm import tqdm
from pathlib import Path
import xarray as xr
import json

# Plotting
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from plotly.colors import n_colors

C:\Users\dekkerm\AppData\Local\Temp\2\ipykernel_1956\3960362660.py:3: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


## Parameter

In [2]:
standard_settings = dict(Risk=0.5,
                         NegEmis=0.5,
                         Convergence_year=2050,
                         NonCO2red=0.5,
                         Timing='Immediate')
params = dict(start_year_analysis=2021)
rules = ['GF', 'PC', 'PCC', 'ECPC', 'AP', 'GDR']
rulecolors = ['steelblue', 'goldenrod', 'forestgreen', 'tomato', 'sienna', 'mediumvioletred']
costoptimal_colors = ['orangered', 'red', 'darkred']

## Paths

In [3]:
path_main = Path("X:/user/dekkerm/Projects/ECEMF_T5.2/")
path_data = Path("K:/ECEMF/T5.2/")
path_figs = path_main / "Figures" / "ECEMF_paper"

## Read data files

In [4]:
xr_dataread = xr.open_dataset(path_data / "xr_dataread.nc")
xr_temps = xr.open_dataset(path_data / "xr_temps.nc")

all_regions_iso = np.load(path_data / "all_regions.npy")
all_regions_names = np.load(path_data / "all_regions_names.npy")
all_countries_iso = np.load(path_data / "all_countries.npy", allow_pickle=True)
all_countries_names = np.load(path_data / "all_countries_names.npy", allow_pickle=True)

## Computations

## Plot

Annotations and helper stuff

In [5]:
annotations = [
    dict(xref='paper', yref='paper', ax=0, align='left', ay=-0, showarrow=False, font=dict(color='black', size=18), y=1.02, x=0.10, text="<b>(a) Grandfathering</b>"),
    dict(xref='paper', yref='paper', ax=0, align='left', ay=-0, showarrow=False, font=dict(color='black', size=18), y=1.02, x=0.74, text="<b>(b) Per Capita</b>"),
    dict(xref='paper', yref='paper', ax=0, align='left', ay=-0, showarrow=False, font=dict(color='black', size=18), y=0.68, x=0.10, text="<b>(c) Per Capita Convergence</b>"),
    dict(xref='paper', yref='paper', ax=0, align='left', ay=-0, showarrow=False, font=dict(color='black', size=18), y=0.68, x=0.91, text="<b>(d) Equal Cumulative per Capita</b>"),
    dict(xref='paper', yref='paper', ax=0, align='left', ay=-0, showarrow=False, font=dict(color='black', size=18), y=0.32, x=0.10, text="<b>(e) Ability to Pay</b>"),
    dict(xref='paper', yref='paper', ax=0, align='left', ay=-0, showarrow=False, font=dict(color='black', size=18), y=0.32, x=0.95, text="<b>(f) Greenhouse Development Rights</b>"),
]

In [6]:
emis_red = (xr_dataread.GHG_hist.sel(Time=params['start_year_analysis']) - xr_dataread.GHG_ndc.sel(Time=2030)) / xr_dataread.GHG_ndc.sel(Time=2030)
emis_red_av = emis_red.mean(dim=['Conditionality', 'Hot_air', 'Ambition']).sel(Region=all_countries_iso)

In [7]:
cty_sort = np.array(emis_red_av.Region)[np.argsort(emis_red_av)]
wh = np.zeros(len(cty_sort))
for cty_i, cty in enumerate(cty_sort):
    if cty in ['USA', 'CHN', 'IND', 'DEU', 'FRA', 'AUS', 'CAN', 'RUS', 'BRA', 'ZAF', 'ARG', 'EGY', 'SAU', 'MEX', 'IDN', 'KAZ', "CHL", "KEN", 'TZA']:
        wh[cty_i] = 1
cty_sort_omit = np.copy(all_countries_names[np.argsort(emis_red_av)])
cty_sort_omit[wh ==0] = ''
cty_sort_omit = np.array([i.replace(' ', '<br>') for i in cty_sort_omit])
#cty_sort_omit[cty_sort_omit == "United States of America"] = 'United States<br>of America'
cols = np.array(['forestgreen']*len(cty_sort))
for cty_i, cty in enumerate(cty_sort):
    try:
        if cont_reg[cont_iso == cty] == 'Western Europe' or cont_reg[cont_iso == cty] == 'Eastern Europe' or cont_reg[cont_iso == cty] == 'Northern Europe' or cont_reg[cont_iso == cty] == 'Southern Europe' or cont_reg[cont_iso == cty] == 'Northern America':
            wh[cty_i] = 1
        elif cont_reg[cont_iso == cty] == 'Southern Asia' or cont_reg[cont_iso == cty] == 'Eastern Asia' or cont_reg[cont_iso == cty] == "Central Asia" or cont_reg[cont_iso == cty] == 'South-eastern Asia' or cont_reg[cont_iso == cty] == 'Western Asia':
            wh[cty_i] = 2
        elif cont_reg[cont_iso == cty] == 'Northern Africa' or cont_reg[cont_iso == cty] == "Sub-Saharan Africa":
            wh[cty_i] = 3
        elif cont_reg[cont_iso == cty] == 'Latin America and the Caribbean':
            wh[cty_i] = 4
    except:
        wh[cty_i] = -1
cols[wh == -1] = 'silver'
cols[wh == 0] = 'silver'
cols[wh == 1] = 'steelblue'
cols[wh == 2] = 'goldenrod'
cols[wh == 3] = 'tomato'
cols[wh == 4] = 'forestgreen'
textpos = np.array(['bottom center']*len(cty_sort))
textpos[wh != 3] = 'top center'
textpos[wh == 4] = 'bottom center'

In [8]:
temp_metric = xr_temps.sel(**standard_settings, Region=all_countries_iso).mean(dim=['Conditionality', 'Hot_air', 'Ambition', 'Scenario'])

In [9]:
fig = make_subplots(rows=3, cols=2, specs=[[{'type': 'choropleth'}, {'type': 'choropleth'}],
                                           [{'type': 'choropleth'}, {'type': 'choropleth'}],
                                           [{'type': 'choropleth'}, {'type': 'choropleth'}]],
                    horizontal_spacing = 0, vertical_spacing=0.0)

for rule_i, rule in enumerate(rules):
    t = np.array(temp_metric[rule])
    fig.add_trace(
        go.Choropleth(
        locations=all_countries_iso,
        z = t,
        locationmode = 'ISO-3',
        colorscale = 'RdYlGn_r',
        zmax = 2.4,
        zmin = 1.4,
        text = [str(r)+": "+str(np.round(float(t[r_i]), 2))+" C" for r_i, r in enumerate(all_countries_iso)],
        hovertemplate  = '%{text}',
        name="",
        marker_line_color='black', 
        marker_line_width=0.5,
        colorbar=dict(len=1, x=1, tickvals = np.arange(1.2, 3, 0.1).round(1), ticktext=['1.2&deg; C',
 '1.3&deg; C',
 '1.4&deg; C',
 '1.5&deg; C',
 '1.6&deg; C',
 '1.7&deg; C',
 '1.8&deg; C',
 '1.9&deg; C',
 '2.0&deg; C',
 '2.1&deg; C',
 '2.2&deg; C',
 '2.3&deg; C',
 '>2.4&deg; C',
 '2.5&deg; C',
 '2.6&deg; C',
 '2.7&deg; C',
 '2.8&deg; C',
 '2.9&deg; C'],
                      tickfont={'size':17}, title={"text":"<b>Temperature<br>rise<br>associated<br>with NDC", 'font': {'size':17}}),
        showscale=[True, False, False, False, False, False][rule_i]
    ), [1, 1, 2, 2, 3, 3][rule_i], [1, 2, 1, 2, 1, 2][rule_i])

fig.update_geos(visible=False,
                showlakes=True,
                lakecolor='rgb(255, 255, 255)',
                projection_type='natural earth',
                resolution=110,
                showcoastlines=False,)

#fig.update_geos(visible=True, showcountries=False)
fig.update_layout(margin={"r":0,"t":20,"l":0,"b":0})

fig.update_layout(
    legend=dict(
        yanchor="top",
        y=0.29,
        xanchor="left",
        x=0.08,
        font=dict(size=15, color="black"),
    ),
    template='ggplot2',
    height=920,
    width=1200,
)
fig.update_xaxes(visible=False)

for ann in annotations:
    fig.add_annotation(**ann)
fig

In [10]:
fig.write_image('X:/user/dekkerm/Projects/EffortSharing/Figures/ECEMF_paper/Figure_3.svg')